In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [ ]:
# Creamos un nuevo dataframe para la info de las ciudades, poblacion y equipos

cities["NHL"] = cities["NHL"].str.replace(r"\[.+\]", "", regex=True)
cities["NHL"] = cities["NHL"].replace({
    "Rangers Islanders Devils": "Rangers,Islanders,Devils",
    "Kings Ducks": "Kings,Ducks"
})
cities["NHL"] = cities["NHL"].apply(lambda x: x.split(','))
cities = cities.explode("NHL")

infocities = cities.iloc[:, [0,1,5]].copy()

infocities.rename(columns={0: 'Metropolitan area', 1: 'Population'}, inplace=True)
infocities.columns = ['Metropolitan area', 'Population', 'NHL']
infocities = infocities[infocities['NHL'].str.contains(r'[a-zA-Z]')]
infocities["NHL"]

# Creamos un dataframe para la info de los equipos y sus victorias y derrotas
info_league = nhl_df[nhl_df['year'] == 2018][['team', 'W', 'L']]
info_league['team'] = info_league['team'].str.replace('*', '')
info_league = info_league[~info_league['team'].str.contains('Division')]
info_league

teams_shorts = infocities["NHL"].str.strip()
teams_complete = info_league["team"].str.strip()

final_list = []

# Verificar si hay coincidencias parciales
for short_name in teams_shorts:
# Encontrar el índice donde se encuentra la coincidencia
    idx = teams_complete[teams_complete.str.contains(short_name)].index
    if not idx.empty:
        index = idx[0]
        #print(f"Match found for '{short_name}', corresponding full team name: {teams_complete[index]}")
        
        # Acceder a la fila correspondiente en info_league
        team_complete_row = info_league.loc[index]

        # Acceder a la fila correspondiente en infocities
        infocities_row = infocities[infocities['NHL'] == short_name].iloc[0]

               # Combinar las filas en una sola con pd.concat()
        combined_row = pd.concat([team_complete_row, infocities_row])

        # Agregar la fila combinada a la lista
        final_list.append(combined_row)

        # Imprimir la fila combinada
        #print(combined_row)
        #print("--------------------------------------------------------")

final_df = pd.DataFrame(final_list)
final_df

result_df = final_df.copy()

result_df['W'] = pd.to_numeric(result_df['W'])
result_df['L'] = pd.to_numeric(result_df['L'])

result_df['Win-Loss Ratio'] = result_df['W'] / (result_df['W'] + result_df['L'])
result_df = result_df.drop(["team", "NHL"], axis=1)

# print(len(result_df.drop_duplicates(subset=["Metropolitan area"]))) # Ya son 28 valores

result_df

# Eliminar filas duplicadas en la columna 'Metropolitan area'
df_unique = result_df.drop_duplicates(subset=['Metropolitan area'])

# Calcular la media del Win-Loss Ratio por cada área metropolitana
mean_ratio = result_df.groupby('Metropolitan area')['Win-Loss Ratio'].mean().reset_index()
mean_ratio

population_data = result_df.groupby('Metropolitan area')['Population'].first().reset_index()
population_data

mean_ratio_with_population = pd.merge(mean_ratio, population_data, on='Metropolitan area', how='left')
mean_ratio_with_population

mean_ratio_with_population['Population'] = mean_ratio_with_population['Population'].astype(float)

population_by_region = mean_ratio_with_population['Population']
win_loss_by_region = mean_ratio_with_population['Win-Loss Ratio'] 

final = stats.pearsonr(population_by_region, win_loss_by_region)[0]
print(final)


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    # Creamos un nuevo dataframe para la info de las ciudades, poblacion y equipos

    cities["NHL"] = cities["NHL"].str.replace(r"\[.+\]", "", regex=True)
    cities["NHL"] = cities["NHL"].replace({
        "Rangers Islanders Devils": "Rangers,Islanders,Devils",
        "Kings Ducks": "Kings,Ducks"
    })
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(','))
    cities = cities.explode("NHL")

    infocities = cities.iloc[:, [0,1,5]].copy()

    infocities.rename(columns={0: 'Metropolitan area', 1: 'Population'}, inplace=True)
    infocities.columns = ['Metropolitan area', 'Population', 'NHL']
    infocities = infocities[infocities['NHL'].str.contains(r'[a-zA-Z]')]
    infocities["NHL"]

    # Creamos un dataframe para la info de los equipos y sus victorias y derrotas
    info_league = nhl_df[nhl_df['year'] == 2018][['team', 'W', 'L']]
    info_league['team'] = info_league['team'].str.replace('*', '')
    info_league = info_league[~info_league['team'].str.contains('Division')]
    info_league

    teams_shorts = infocities["NHL"].str.strip()
    teams_complete = info_league["team"].str.strip()

    final_list = []

    # Verificar si hay coincidencias parciales
    for short_name in teams_shorts:
    # Encontrar el índice donde se encuentra la coincidencia
        idx = teams_complete[teams_complete.str.contains(short_name)].index
        if not idx.empty:
            index = idx[0]
            #print(f"Match found for '{short_name}', corresponding full team name: {teams_complete[index]}")

            # Acceder a la fila correspondiente en info_league
            team_complete_row = info_league.loc[index]

            # Acceder a la fila correspondiente en infocities
            infocities_row = infocities[infocities['NHL'] == short_name].iloc[0]

                   # Combinar las filas en una sola con pd.concat()
            combined_row = pd.concat([team_complete_row, infocities_row])

            # Agregar la fila combinada a la lista
            final_list.append(combined_row)

            # Imprimir la fila combinada
            #print(combined_row)
            #print("--------------------------------------------------------")

    final_df = pd.DataFrame(final_list)
    final_df

    result_df = final_df.copy()

    result_df['W'] = pd.to_numeric(result_df['W'])
    result_df['L'] = pd.to_numeric(result_df['L'])

    result_df['Win-Loss Ratio'] = result_df['W'] / (result_df['W'] + result_df['L'])
    result_df = result_df.drop(["team", "NHL"], axis=1)

    # print(len(result_df.drop_duplicates(subset=["Metropolitan area"]))) # Ya son 28 valores

    result_df

    # Eliminar filas duplicadas en la columna 'Metropolitan area'
    df_unique = result_df.drop_duplicates(subset=['Metropolitan area'])

    # Calcular la media del Win-Loss Ratio por cada área metropolitana
    mean_ratio = result_df.groupby('Metropolitan area')['Win-Loss Ratio'].mean().reset_index()
    mean_ratio

    population_data = result_df.groupby('Metropolitan area')['Population'].first().reset_index()
    population_data

    mean_ratio_with_population = pd.merge(mean_ratio, population_data, on='Metropolitan area', how='left')
    mean_ratio_with_population

    mean_ratio_with_population['Population'] = mean_ratio_with_population['Population'].astype(float)

    population_by_region = mean_ratio_with_population['Population']
    win_loss_by_region = mean_ratio_with_population['Win-Loss Ratio'] 

    final = stats.pearsonr(population_by_region, win_loss_by_region)[0]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return final

nhl_correlation()

0.012486162921209923